# **Selamat Datang di Proyek Data Analis Pertama!)**

Kita akan mencoba melakukan eksplorasi data menggunakan dataset **Titanic** dari Kaggle — ini adalah salah satu dataset paling populer untuk latihan Data Science.

Dataset dapat kamu akses di:

> [https://www.kaggle.com/c/titanic-gettingStarted](https://www.kaggle.com/c/titanic-gettingStarted)

Silakan unduh berkas **train.csv**, lalu simpan pada direktori yang sama dengan notebook Python-mu.

---

## **1. Membuka Dataset dengan pandas**

```python
import pandas as pd
from pandas import Series, DataFrame

titanic_df = pd.read_csv('train.csv')
titanic_df.head()
```



In [3]:
import pandas as pd
from pandas import Series, DataFrame

titanic_df = pd.read_csv('train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Output menunjukkan beberapa baris awal, berisi informasi seperti:

* **Survived**: 0 = meninggal, 1 = selamat
* **Pclass**: kelas 1, 2, atau 3
* **Sex**: jenis kelamin
* **Age**: umur
* **Fare**, **Cabin**, **Embarked**, dll.

Untuk mengetahui struktur keseluruhan dataset:

```python
titanic_df.info()
```

Analisis:

* Ada **891 baris** (penumpang).
* Kolom **Age** hanya memiliki **714 nilai** → ada banyak **missing values**.
* Kolom **Cabin** hanya memiliki 204 nilai → sebagian besar penumpang tidak diketahui kabinnya.
* Tipe data campuran: numerik dan kategorikal.

---

# **Pertanyaan Analisis**

Sebelum menganalisis, kita tentukan dulu apa yang ingin kita cari. Beberapa pertanyaan eksplorasi yang umum:

### **Pertanyaan Dasar**

1. Siapa saja penumpang Titanic? (umur, gender, kelas, dll.)
2. Di dek mana penumpang berada, dan apakah ini berhubungan dengan kelas?
3. Dari mana penumpang berasal? (asal embarkasi)
4. Siapa yang bepergian sendirian dan siapa yang bersama keluarga?

### **Pertanyaan Lebih Lanjut**

5. Faktor apa saja yang mempengaruhi peluang seseorang untuk selamat?

Mari kita mulai.

---

# **2. Siapa Penumpang Titanic? (Exploratory Data Analysis)**

Kita akan menggunakan **matplotlib**, **numpy**, dan **seaborn** untuk visualisasi:

```python
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
```

## **A. Distribusi Gender**

```python
sns.factorplot('Sex', data=titanic_df)
```

Analisis:

* Penumpang laki-laki **jauh lebih banyak** dibanding perempuan.
* Ini penting karena kebijakan "women and children first" dapat memengaruhi survival rate.

## **B. Gender berdasarkan Kelas**

```python
sns.factorplot('Pclass', data=titanic_df, hue='Sex')
```

Analisis:

* **Kelas 3** memiliki jumlah laki-laki paling banyak → implikasi terhadap survival.

## **C. Menambahkan Kategori “Child”**

Orang di bawah 16 tahun akan dikategorikan sebagai *child*.

```python
def male_female_child(passenger):
    age, sex = passenger
    if age < 16:
        return 'child'
    else:
        return sex

titanic_df['person'] = titanic_df[['Age','Sex']].apply(male_female_child, axis=1)
```

Lalu kita lihat distribusinya:

```python
sns.factorplot('Pclass', data=titanic_df, hue='person')
```

Analisis:

* Anak-anak paling banyak berada di **kelas 3**.
* Hampir tidak ada anak dalam **kelas 1**.

## **D. Distribusi Umur**

```python
titanic_df['Age'].hist(bins=70)
```

Analisis:

* Banyak penumpang berusia 20–40 tahun.
* Distribusi agak miring ke kanan (lebih banyak penumpang muda).

## **E. KDE Plot untuk Perbandingan Umur**

Dengan seaborn FacetGrid:

```python
fig = sns.FacetGrid(titanic_df, hue="Sex", aspect=4)
fig.map(sns.kdeplot, 'Age', shade=True)
fig.set(xlim=(0, titanic_df['Age'].max()))
fig.add_legend()
```

Analisis:

* Laki-laki mendominasi hampir di semua rentang usia.

Kita dapat melakukan hal sama untuk “person” dan “Pclass”.

---

# **3. Di Dek Mana Penumpang Berada?**

Kolom **Cabin** seperti “C85”, “E46”, dll.
Kita hanya membutuhkan **huruf pertama** untuk mengetahui dek.

```python
deck = titanic_df['Cabin'].dropna()

levels = [c[0] for c in deck]

cabin_df = DataFrame(levels)
cabin_df.columns = ['Cabin']

sns.factorplot('Cabin', data=cabin_df)
```

Analisis:

* Dek **C** dan **B** paling umum. Ini umumnya kelas 1.
* Dek **G** jarang, kemungkinan kelas rendah.
* Ada satu nilai “T” → error/noise → dibuang.

---

# **4. Dari Mana Penumpang Berasal?**

Kolom **Embarked**:

* C = Cherbourg
* Q = Queenstown
* S = Southampton

Visualisasi:

```python
sns.factorplot('Embarked', data=titanic_df, hue='Pclass', x_order=['C','Q','S'])
```

Analisis:

* Penumpang dari **Queenstown** hampir semuanya kelas 3.
* Southampton memiliki penumpang terbanyak.

---

# **5. Siapa yang Berpergian Sendiri?**

Buat kolom baru:

```python
titanic_df['Alone'] = titanic_df.Parch + titanic_df.SibSp
titanic_df['Alone'].loc[titanic_df['Alone'] > 0] = 'With Family'
titanic_df['Alone'].loc[titanic_df['Alone'] == 0] = 'Alone'
```

Visualisasi:

```python
sns.factorplot('Alone', data=titanic_df)
```

Analisis:

* Banyak penumpang **pergi sendirian**.
* Ini akan berpengaruh ke survival (keluarga lebih mungkin membantu).

---

# **6. Faktor yang Mempengaruhi Survival**

Buat kolom readable:

```python
titanic_df["Survivor"] = titanic_df.Survived.map({0:"no",1:"yes"})
```

### **A. Distribusi Selamat vs Tidak**

```python
sns.factorplot('Survivor', data=titanic_df)
```

Analisis:

> Yang meninggal **lebih banyak** dari yang selamat.

### **B. Survival berdasarkan Kelas**

```python
sns.factorplot('Pclass', 'Survived', data=titanic_df)
```

Analisis:

* Kelas 1 → peluang selamat paling tinggi.
* Kelas 3 → paling rendah.

### **C. Kombinasi Kelas + Gender**

```python
sns.factorplot('Pclass','Survived', hue='person', data=titanic_df)
```

Analisis:

* **Perempuan dan anak-anak** memiliki peluang selamat tertinggi, mengikuti kebijakan kapal.
* Laki-laki kelas berapa pun → peluang kecil.

### **D. Pengaruh Umur**

```python
sns.lmplot('Age', 'Survived', data=titanic_df)
```

Analisis:

* Semakin tua, peluang selamat menurun.

### **E. Umur + Kelas**

```python
sns.lmplot('Age','Survived', hue='Pclass', data=titanic_df)
```

Analisis:

* Penumpang tua kelas 1 memiliki peluang lebih baik dibanding kelas lain.

### **F. Umur + Gender**

```python
sns.lmplot('Age','Survived', hue='Sex', data=titanic_df)
```

Analisis:

* Perempuan dari semua usia memiliki peluang lebih tinggi untuk selamat.

---

# **Tugas / Pertanyaan untuk Dilanjutkan**

1. Apakah dek berpengaruh terhadap peluang selamat?
2. Apakah bepergian bersama keluarga meningkatkan peluang selamat?

Keduanya bisa dieksplorasi dengan seaborn (factorplot, barplot, lmplot) atau analisis statistik sederhana.

---

Jika kamu mau, aku bisa lanjut:

✅ membuat jawaban atas pertanyaan 1–2
✅ membuat versi notebook (.ipynb)
✅ membuat versi ringkas untuk materi kuliah
✅ menambahkan interpretasi statistik (chi-square, logistic regression)

Cukup bilang: **“lanjut”** atau minta format lain.
